In [1]:
# Install required packages
!pip install --upgrade pip
!pip install ultralytics opencv-python kagglehub huggingface-hub pillow torch torchvision pyyaml

# Verify installations
import ultralytics
print(f"YOLOv8 version: {ultralytics.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLOv8 version: 8.4.12


In [4]:
import os

# Change this path to the location of your dataset on your desktop
dataset_path = "/Users/Vinay/Desktop/screendetectionDataset"


print("\nLoading dataset from local desktop...")

if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset not found at {dataset_path}. Please check the path.")

print(f"\nDataset found at: {dataset_path}")

# List dataset contents
print("\nDataset contents:")
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        file_count = len(os.listdir(item_path))
        print(f"  {item}: {file_count} files")
    else:
        print(f"  {item}")



Loading dataset from local desktop...


FileNotFoundError: Dataset not found at /Users/Vinay/Desktop/evalsystem-go. Please check the path.

In [ ]:
import os
import random
import shutil
from pathlib import Path
import xml.etree.ElementTree as ET

# ---------------- CONFIG ----------------
SOURCE_DIR = r"/root/.cache/kagglehub/datasets/dataclusterlabs/handbag-image-dataset-luggage-dataset/versions/1/hand bag 300"
# folder with jpg + xml together
YOLO_ROOT = "/content/yolo_dataset"
TRAIN_RATIO = 0.8

CLASSES = {
    "hand_bag": 0
}

# ----------------------------------------

def convert_voc_to_yolo(xml_path, txt_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find("size")
    img_w = float(size.find("width").text)
    img_h = float(size.find("height").text)

    lines = []

    for obj in root.findall("object"):
        cls_name = obj.find("name").text.strip()

        if cls_name not in CLASSES:
            continue

        cls_id = CLASSES[cls_name]
        bnd = obj.find("bndbox")

        xmin = float(bnd.find("xmin").text)
        ymin = float(bnd.find("ymin").text)
        xmax = float(bnd.find("xmax").text)
        ymax = float(bnd.find("ymax").text)

        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        lines.append(
            f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        )

    with open(txt_path, "w") as f:
        f.write("\n".join(lines))


# ---------------- CREATE DIRS ----------------
for split in ["train", "val"]:
    os.makedirs(f"{YOLO_ROOT}/images/{split}", exist_ok=True)
    os.makedirs(f"{YOLO_ROOT}/labels/{split}", exist_ok=True)

# ---------------- FIND FILES ----------------
images = sorted([f for f in os.listdir(SOURCE_DIR) if f.lower().endswith(".jpg")])
random.shuffle(images)

split_idx = int(len(images) * TRAIN_RATIO)
train_imgs = images[:split_idx]
val_imgs = images[split_idx:]

def process_split(image_list, split):
    for img_name in image_list:
        base = Path(img_name).stem
        img_src = f"{SOURCE_DIR}/{img_name}"
        xml_src = f"{SOURCE_DIR}/{base}.xml"

        if not os.path.exists(xml_src):
            print(f"⚠️ Missing XML for {img_name}")
            continue

        img_dst = f"{YOLO_ROOT}/images/{split}/{img_name}"
        txt_dst = f"{YOLO_ROOT}/labels/{split}/{base}.txt"

        shutil.copy(img_src, img_dst)
        convert_voc_to_yolo(xml_src, txt_dst)

process_split(train_imgs, "train")
process_split(val_imgs, "val")

print("✅ VOC → YOLO conversion complete")
print(f"Train images: {len(train_imgs)}")
print(f"Val images: {len(val_imgs)}")


In [ ]:
import os

def print_tree(path, indent=0, max_depth=4):
    if indent > max_depth:
        return
    for item in sorted(os.listdir(path)):
        full = os.path.join(path, item)
        print("  " * indent + "├── " + item)
        if os.path.isdir(full):
            print_tree(full, indent + 1)

print_tree("/content/yolo_dataset")


In [ ]:
# Path where the YAML file will be created
yaml_path = "/content/handbag.yaml"

yaml_content = """
path: /content/yolo_dataset

train: images/train
val: images/val

nc: 1
names:
  - hand_bag
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

print(f"✅ YOLO dataset YAML created at: {yaml_path}")


In [ ]:
from huggingface_hub import login
import getpass

print("\n" + "="*60)
print("🤗 HUGGING FACE AUTHENTICATION")

hf_token ="xxxxx"

# Login to Hugging Face
try:
    login(token=hf_token)
    print("\n  Logged in to Hugging Face successfully!")
except Exception as e:
    print(f"\n  Authentication failed: {str(e)}")
    raise



from huggingface_hub import hf_hub_download
from ultralytics import YOLO

REPO_ID = "IndUSV/Yolov8_SE_2"
FILENAME = "Yolov8_SE_2.pt"   # ⚠️ must be a YOLO .pt file

model_path = hf_hub_download(
    repo_id=REPO_ID,
    filename=FILENAME
)

print("✅ Downloaded model to:", model_path)



In [ ]:
from ultralytics import YOLO

model = YOLO(model_path)

print("\n" + "="*60)
print("STARTING MODEL TRAINING")
print("="*60 + "\n")

epochs = 50
imgsz = 640
batch_size = 16
patience = 10

print("Training configuration:")
print(f"  • Model: YOLOv8n")
print(f"  • Epochs: {epochs}")
print(f"  • Image size: {imgsz}")
print(f"  • Batch size: {batch_size}")
print(f"  • Early stopping patience: {patience}")
print(f"  • Dataset YAML: {yaml_path}\n")

results = model.train(
    data=yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    patience=patience,
    device="cpu",
    project="hand_bag_detector",
    name="yolov8n_v1",
    exist_ok=False,
    save=True,
    save_period=5,
    plots=True,
    verbose=True
)

print("\n" + "="*60)
print("TRAINING COMPLETED")
print("="*60)


In [ ]:
from huggingface_hub import login
import getpass

print("\n" + "="*60)
print("🤗 HUGGING FACE AUTHENTICATION")

hf_token ="xxxxxx"

# Login to Hugging Face
try:
    login(token=hf_token)
    print("\n  Logged in to Hugging Face successfully!")
except Exception as e:
    print(f"\n  Authentication failed: {str(e)}")
    raise

In [ ]:
from huggingface_hub import HfApi
from pathlib import Path

# =========================
# CONFIGURATION
# =========================

REPO_ID = "IndUSV/Yolov8_SE_3"

BEST_MODEL_PATH = Path(
    "/content/runs/detect/hand_bag_detector/yolov8n_v1/weights/best.pt"
)

# =========================
# INIT HF API
# =========================

api = HfApi()

print("\n" + "=" * 60)
print(" PUSHING TRAINED MODEL TO HUGGING FACE")
print("=" * 60)
print(f"\nTarget repository: {REPO_ID}\n")

# =========================
# CREATE / VERIFY REPO
# =========================

api.create_repo(
    repo_id=REPO_ID,
    repo_type="model",
    exist_ok=True,
    private=False
)

print("Repository ready")

# =========================
# CHECK MODEL
# =========================

if not BEST_MODEL_PATH.exists():
    raise FileNotFoundError(f"Model not found: {BEST_MODEL_PATH}")

print(f"Model found: {BEST_MODEL_PATH}")
print(f"Size: {BEST_MODEL_PATH.stat().st_size / (1024*1024):.2f} MB\n")

# =========================
# UPLOAD MODEL (CORRECT NAME)
# =========================

print("Uploading model weights...")

api.upload_file(
    path_or_fileobj=str(BEST_MODEL_PATH),
    path_in_repo="Yolov8_SE_2.pt",
    repo_id=REPO_ID,
    repo_type="model"
)

print("✅ Model uploaded successfully!")



In [ ]:
from huggingface_hub import HfApi
from datetime import datetime

model_card_content = f"""
---
license: mit
tags:
- yolov8
- object-detection
- computer-vision
- mobile-phone
- suitcase
---

# YOLOv8n Multi-Object Detector (Phone & Suitcase)

A fine-tuned **YOLOv8 Nano (YOLOv8n)** model capable of detecting **mobile phones**, **suit cases** and **handbags**  in images.
The model has been trained and validated on custom datasets with diverse backgrounds, lighting conditions,
and object orientations.

## Model Details

- **Base Model**: YOLOv8n (Nano)
- **Task**: Object Detection
- **Supported Classes**:
  - `mobile_phone`
  - `suitcase`
- **Number of Classes**: 2
- **Input Resolution**: 640 × 640
- **Framework**: Ultralytics YOLO
- **Training Date**: {datetime.now().strftime('%Y-%m-%d')}

## Capabilities

This model can:
- Detect **mobile phones** in real-world scenes
- Detect **suitcases / luggage** in travel and indoor environments
- Handle multiple objects per image
- Perform robustly under varied lighting and viewpoints

## Training Details

- **Image Size**: 640
- **Optimizer**: Default YOLOv8 optimizer
- **Augmentations**: YOLOv8 default augmentations
- **Device**: NVIDIA GPU
- **Training Strategy**: Transfer learning from YOLOv8n

## Dataset Information

The model was trained using curated datasets containing:
- Mobile phone images with bounding-box annotations
- Suitcase / luggage images with bounding-box annotations
- Real-world indoor and outdoor scenes
- Multiple object instances per image

Dataset annotations follow object-detection standards compatible with YOLO training pipelines.

## Evaluation & Metrics

- Quantitative evaluation metrics are available in `results.csv`
- Qualitative predictions can be seen in validation images from the training run

## Files in This Repository

- `Yolov8_SE_2.pt` – Trained YOLOv8 model weights
- `args.yaml` – Training configuration
- `results.csv` – Training and validation metrics

## License

This model is released under the **MIT License**.

## Acknowledgements

- Ultralytics YOLOv8: https://github.com/ultralytics/ultralytics
- Training datasets sourced and curated for object detection research
"""

# Upload model card
api = HfApi()

print("\n  Creating and uploading model card...")
api.upload_file(
    path_or_fileobj=model_card_content.encode("utf-8"),
    path_in_repo="README.md",
    repo_id=REPO_ID,
    repo_type="model"
)

print("\n" + "=" * 60)
print("  MODEL CARD UPDATED SUCCESSFULLY")
print("=" * 60)
print(f"\n  Model URL: https://huggingface.co/{REPO_ID}")
print("  Capabilities: Mobile Phone Detection + Suitcase Detection")
